In [2]:
from ts_vae.data_processors.ts_gen_processor import TSGenDataset
from torch_geometric.data import DataLoader
import numpy as np
from ts_vae.utils import remove_files
remove_files() 
rxns = TSGenDataset(r'data')
rxns[0]
###
tt_split = 0.8
num_rxns = len(rxns)
num_train = int(np.floor(tt_split * num_rxns))
batch_size = 2
train_loader = DataLoader(rxns[: num_train], batch_size = batch_size)
test_loader = DataLoader(rxns[num_train: ], batch_size = batch_size)
###
batch = next(iter(train_loader))
test = batch.edge_attr

Files removed.
Processing...


  0%|          | 10/7581 [00:00<02:23, 52.77it/s]


Done!


In [3]:
batch.num_atoms

tensor([15, 13])

In [24]:
from ts_vae.data_processors.ts_gen_processor import TSGenDataset
import torch
from torch_geometric.data import DataLoader
import tqdm
rxns = TSGenDataset(r'data')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # NOTE

train_loader = DataLoader(rxns[: 300], batch_size = 8)

for inputs in train_loader:
    inputs = inputs.to(device)

# train_loader.is_cuda

In [5]:
from experiments.building_on_mit.meta_eval.meta_eval import ablation_experiment
# from ts_vae.utils import remove_files
# remove_files()
train_log, test_log = ablation_experiment(tt_split = 0.8, epochs = 1, test_interval = 1, \
                                          batch_size = 8, h_nf = 256, n_layers = 2, gnn_depth = 3)

# b=8, h=256, gnn_depth=3, n_layers=2
# b=20, h=256, n_layers=2, gnn_depth=3 -> 33, 36; e=125s
# b=20, h=256, n_layers=1, gnn_depth=3 -> 58, 36; e=50s
# b=20, h=256, n_layers=1, gnn_depth=2 -> 35, 36; e=70s

Starting ablation experiment...
===== Training epoch 001 complete with loss: 2.3476 ====
===== Testing epoch 001 complete with loss: 2.2618 ====
Completed ablation experiment, use the experiment log to print results.


In [2]:
import torch

# want to create a D_init of [num_train, 21, 21] for plotting
batched_test_Ds = test_log.epoch_ae_results[0]['D_init']
test_Ds = torch.cat([D for D in batched_test_Ds], 0)
test_Ds = test_Ds.detach().cpu().numpy()

# might be easier to view training dynamics this way

test_Ds.shape

(200, 21, 21)